# Import libraries


In [74]:
import pandas as pd
import plotly.express as px
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display

# Data Load

In [75]:
# Load the CSV file into a DataFrame
file_path = '/content/2024_Q1.csv'  # Update with the correct path if needed
data = pd.read_csv(file_path)

# Data Engineering

In [76]:
# Step 1: Convert date columns to datetime format
date_columns = ['checkin_date', 'checkout_date', 'booking_date']
for col in date_columns:
    data[col] = pd.to_datetime(data[col])

# Step 2: Handle missing data
# Dropping columns with a significant amount of missing data (e.g., host_last_name, review_id)
data = data.drop(columns=['host_last_name', 'review_id', 'review_rating'])

# Calculate KPIs

In [77]:
def calculate_kpis(data):

    # Calculate total nights and total revenue
    total_nights = data['nights'].sum()
    total_revenue = data['revenue'].sum()

    # Calculate total available nights
    num_properties = data['property_name'].nunique()
    period_days = (data['checkout_date'].max() - data['checkin_date'].min()).days + 1
    total_available_nights = period_days * num_properties

    # Calculate KPIs
    occupancy_rate = round((total_nights / total_available_nights) * 100, 2) if total_available_nights else 0
    adr = round((total_revenue / total_nights), 2) if total_nights else 0
    revpar = round((total_revenue / total_available_nights), 2) if total_available_nights else 0

    # Calculate average lead time in days
    lead_time = round((data['checkin_date'] - data['booking_date']).dt.days.mean(), 2)

    # Calculate the number of reservations
    number_of_reservations = data.shape[0]

    # Calculate the average length of stay
    average_length_of_stay = round(data['nights'].mean(), 2) if number_of_reservations else 0

    # Calculate Cancellation Rate
    total_reservations = data.shape[0]
    canceled_reservations = data[data['status'] == 'cancelled'].shape[0]
    cancellation_rate = round((canceled_reservations / total_reservations) * 100, 2)

    # Return the KPIs as a dictionary
    return {
        'Occupancy Rate (%)': f"{occupancy_rate}%",
        'Average Daily Rate (ADR)': f"€{adr}",
        'Revenue Per Available Room (RevPAR)': f"€{revpar}",
        'Total Revenue': f"€{total_revenue:.2f}",
        'Average Lead Time (days)': f"{lead_time}",
        'Number of Reservations': number_of_reservations,
        'Average Length of Stay (nights)': average_length_of_stay,
        'Cancellation Rate (%)': cancellation_rate
    }

In [19]:
# Example of using the function
kpis = calculate_kpis(data)
print(kpis)

{'Occupancy Rate (%)': '91.91%', 'Average Daily Rate (ADR)': '€91.01', 'Revenue Per Available Room (RevPAR)': '€83.65', 'Total Revenue': '€39316.82', 'Average Lead Time (days)': '27.87', 'Number of Reservations': 158, 'Average Length of Stay (nights)': 2.73, 'Cancellation Rate (%)': 6.33}


# Viz

## By Channel

In [81]:
def plot_revenue_percentage_per_platform(data):
    revenue_per_platform = data.groupby('platform')['revenue'].sum().reset_index()
    fig = px.pie(revenue_per_platform, values='revenue', names='platform', title='Percentage of Revenue per Platform',
                 color='platform', color_discrete_map={'airbnb': 'red', 'booking.com': 'blue', 'vrbo': 'purple'},
                 hole=0.3)
    return fig

In [13]:
plot_revenue_percentage_per_platform(data)

In [82]:
def plot_nights_booked_percentage_per_platform(df):
    nights_per_platform = df.groupby('platform')['nights'].sum().reset_index()
    fig = px.pie(nights_per_platform, values='nights', names='platform', title='Percentage of Nights Booked per Platform',
                 color='platform', color_discrete_map={'airbnb': 'red', 'booking.com': 'blue', 'vrbo': 'purple'},
                 hole=0.3)
    return fig


In [17]:
plot_nights_booked_percentage_per_platform(data)

## By Channel and Property

In [83]:
def plot_stacked_revenue_by_property_platform(df):
    # Step 1: Aggregate the data
    aggregated_df = df.groupby(['property_name', 'platform'], as_index=False)['revenue'].sum()

    # Step 2: Create the stacked bar chart
    fig = px.bar(aggregated_df, x='property_name', y='revenue', color='platform',
                 title='Stacked Revenue of Property by Platform',
                 color_discrete_map={'airbnb': 'red', 'booking.com': 'blue', 'vrbo': 'purple'})

    return fig

In [23]:
plot_stacked_revenue_by_property_platform(data)

## Properties Comparison Over Time

### Revenue

In [84]:
def plot_monthly_revenue_line(df):
    # Ensure date columns are in datetime format
    df['checkin_date'] = pd.to_datetime(df['checkin_date'])
    df['checkout_date'] = pd.to_datetime(df['checkout_date'])

    # Calculate the start of the month for grouping
    df['month'] = df['checkin_date'].dt.to_period('M').dt.start_time  # Convert to the first day of the month

    # Calculate the monthly revenue per property
    monthly_revenue = df.groupby(['month', 'property_name'])['revenue'].sum().reset_index()

    # Create the line chart
    fig = px.line(monthly_revenue, x='month', y='revenue', color='property_name', title='Monthly Revenue per Property')

    # Update y-axis to start at 0
    fig.update_yaxes(title='Revenue (€)', range=[0, monthly_revenue['revenue'].max() * 1.1])

    # Format x-axis to show month names and ensure chronological order
    fig.update_xaxes(
        title='Month',
        dtick="M1",
        tickformat="%B %Y",  # Show month name and year
        categoryorder="category ascending"  # Ensure chronological order
    )

    return fig

In [66]:
plot_monthly_revenue_line(data)

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



### Occupancy

In [88]:
def plot_monthly_occupancy_rate(df):
    # Ensure date columns are in datetime format
    df.loc[:, 'checkin_date'] = pd.to_datetime(df['checkin_date'])
    df.loc[:, 'checkout_date'] = pd.to_datetime(df['checkout_date'])

    # Calculate the start of the month for grouping
    df.loc[:, 'month'] = df['checkin_date'].dt.to_period('M').dt.start_time

    # Filter cancelled reservations
    filtered_data = data[data['status'] != 'cancelled']

    # Calculate occupancy for each property each month
    monthly_occupancy = df.groupby(['month', 'property_name']).apply(
        lambda x: x['nights'].sum() / ((x['checkout_date'].max() - x['checkin_date'].min()).days + 1)
    ).reset_index(name='occupancy_rate')

    # Create the line chart
    fig = px.line(monthly_occupancy, x='month', y='occupancy_rate', color='property_name',
                  title='Monthly Occupancy Rate per Property')

    # Update y-axis to start at 0
    fig.update_yaxes(title='Occupancy Rate', tickformat=".0%", range=[0, 1])

    # Format x-axis to show month names and ensure chronological order
    fig.update_xaxes(
        title='Month',
        dtick="M1",
        tickformat="%B %Y",  # Show month name and year
        categoryorder="category ascending"  # Ensure chronological order
    )

    return fig



In [89]:
#filtered_data = data[data['status'] != 'cancelled']
fig = plot_monthly_occupancy_rate(filtered_data)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



### ADR

In [86]:
import pandas as pd
import plotly.express as px

def plot_monthly_adr_line(df):
    # Ensure date columns are in datetime format
    df['checkin_date'] = pd.to_datetime(df['checkin_date'])
    df['checkout_date'] = pd.to_datetime(df['checkout_date'])

    # Calculate the start of the month for grouping
    df['month'] = df['checkin_date'].dt.to_period('M').dt.start_time  # Convert to the first day of the month

    # Calculate the total revenue and total nights per month per property
    monthly_data = df.groupby(['month', 'property_name']).agg({
        'revenue': 'sum',
        'nights': 'sum'
    }).reset_index()

    # Calculate ADR (Average Daily Rate)
    monthly_data['adr'] = monthly_data['revenue'] / monthly_data['nights']

    # Create the line chart
    fig = px.line(monthly_data, x='month', y='adr', color='property_name', title='Monthly ADR (Average Daily Rate) per Property')

    # Update y-axis to start at 0
    fig.update_yaxes(title='ADR (€)', range=[0, monthly_data['adr'].max() * 1.1])

    # Format x-axis to show month names and ensure chronological order
    fig.update_xaxes(
        title='Month',
        dtick="M1",
        tickformat="%B %Y",  # Show month name and year
        categoryorder="category ascending"  # Ensure chronological order
    )

    return fig

In [68]:
plot_monthly_adr_line(data)

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



# Dashboard